## Python Libraries und Settings

In [11]:
# Libraries
import os
import sqlite3
import pandas as pd
from sqlalchemy import create_engine, text

# Settings
import warnings
warnings.filterwarnings("ignore")

# Working directory
print(os.getcwd())

/


#### Erstellen und Testen einer Datenbankverbindung mit der 'postgres' Datenbank.

In [12]:
# Set up für Datenbankverbindung
user = "pgadmin"
password = "geheim"
host = "localhost"
port = "5432"
database = "postgres"

# Erstellen der Connection URL
db_connection_url = "postgresql://" + user + ":" + password +\
                    "@" + host + ":" + port + "/" + database

# Erstellen SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Test der Datenbankverbindung
with engine.connect() as connection:
    result = connection.execute(text('SELECT current_database()'))
    print(result.fetchone())

# Verbindung trennen
engine.dispose()

('postgres',)


#### Einlesen der Tabellenblätter aus 'car_dealership_data.xlsx', und speichern dieser in separaten DataFrames.

In [13]:
# Pfad zur .xlsx Datei
path = 'workspace/car_dealer_data.xlsx'

# Einlesen der Daten aus Excel File
df_cars = pd.read_excel(path, sheet_name='Cars')
df_customers = pd.read_excel(path, sheet_name='Customers')
df_sales = pd.read_excel(path, sheet_name='Sales')
df_salespersons = pd.read_excel(path, sheet_name='SalesPersons')
df_mechanics = pd.read_excel(path, sheet_name='Mechanics')
df_servicerecords = pd.read_excel(path, sheet_name='ServiceRecords')

# Anpassen der Spaltennamen zu 'lowercase'
df_cars.columns = df_cars.columns.str.lower()
df_customers.columns = df_customers.columns.str.lower()
df_sales.columns = df_sales.columns.str.lower()
df_salespersons.columns = df_salespersons.columns.str.lower()
df_mechanics.columns = df_mechanics.columns.str.lower()
df_servicerecords.columns = df_servicerecords.columns.str.lower()


#### Schreiben der erstellten DataFrames in separate Tabellen in Datenbank.

In [14]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Schreiben der DataFrames in die Datenbank
df_cars.to_sql('cars', engine, if_exists='replace')
df_customers.to_sql('customers', engine, if_exists='replace')
df_sales.to_sql('sales', engine, if_exists='replace')
df_salespersons.to_sql('salespersons', engine, if_exists='replace')
df_mechanics.to_sql('mechanics', engine, if_exists='replace')
df_servicerecords.to_sql('servicerecords', engine, if_exists='replace')

# Datenbankverbindung trennen
engine.dispose()

#### <b>Aufgabe (1): Ermitteln Sie die Anzahl der durchgeführten Services mit ServiceDescription = 'Clutch Repair'.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Informationen in der Tabelle 'ServiceRecords'.
- In der Ergebnistabelle muss die Anzahl der Services mit ServiceDescription = 'Clutch Repair' als ein Wert erscheinen.



In [15]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Abfrage erstellen und in DataFrame speichern
df_sub = pd.read_sql_query("""select
	count(serviceid) as servicecount_clutchrepair
from servicerecords
WHERE servicedescription in ('Clutch Repair');""",
                          con=engine)

# Datenbankverbindung trennen
engine.dispose()

# Ergebnis anzeigen (Dataframe -Auschnitt genügt- mit Ergebnis der Abfrage)
df_sub

,servicecount_clutchrepair
0,314


#### <b>Aufgabe (2): Ermitteln Sie den mittleren Preis aller Fahrzeuge nach Baujahr.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Informationen in der Tabelle 'Cars'.
- Die Ergebnistabelle muss zwei Spalten beinhalten (Year, AveragePrice).



In [16]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Abfrage erstellen und in DataFrame speichern
df_sub = pd.read_sql_query("""select 
	year,
	AVG(price) as averageprice
from cars
group by year, price;""",
                          con=engine)

# Datenbankverbindung trennen
engine.dispose()

# Ergebnis anzeigen (Dataframe -Auschnitt genügt- mit Ergebnis der Abfrage)
df_sub

,year,averageprice
0,2020,33286.46
1,2021,49783.35
2,2023,19126.05
3,2022,15794.32
4,2022,23809.87
...,...,...
1994,2021,49029.86
1995,2020,32980.97
1996,2022,34150.28
1997,2019,35885.86


#### <b>Aufgabe (3): Listen Sie die Anzahl Fahrzeuge für jede Kombination aus Marke und Modell auf.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Informationen in der Tabelle 'Cars'.
- Jede Marke und jedes Modell darf zusammen mit der entsprechenden Anzahl der Fahrzeuge nur einmal gezeigt werden.
- Die Ergebnistabelle muss drei Spalten beinhalten (Make, Model, NumberOfCars).
- Sortieren Sie die Ergebnistabelle absteigend nach der Anzahl der Fahrzeuge.



In [17]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Abfrage erstellen und in DataFrame speichern
df_sub = pd.read_sql_query("""SELECT distinct
  make,
  model,
  COUNT(*) AS NumberofCars
FROM cars
GROUP BY make, model
order by NumberofCars DESC;""",
                          con=engine)

# Datenbankverbindung trennen
engine.dispose()

# Ergebnis anzeigen (Dataframe -Auschnitt genügt- mit Ergebnis der Abfrage)
df_sub

,make,model,numberofcars
0,Chevrolet,Malibu,179
1,Ford,Fiesta,179
2,Toyota,Camry,179
3,BMW,320i,178
4,Mercedes,E-Class,175
5,Ford,Mustang,164
6,Honda,Accord,163
7,Mercedes,C-Class,161
8,Toyota,Corolla,159
9,Chevrolet,Impala,158


#### <b>Aufgabe (4): Ermitteln Sie die Anzahl der Fahrzeuge mit dem Status 'Available', gruppiert nach Marke.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Informationen in der Tabelle 'Cars'.
- Die Ergebnistabelle muss nach der Anzahl Fahrzeuge absteigend sortiert sein.



In [18]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Abfrage erstellen und in DataFrame speichern
df_sub = pd.read_sql_query("""select 
	make,
	count(status) as availablecars
from cars
where status in ('Available')
group by make
order by availablecars DESC;""",
                          con=engine)

# Datenbankverbindung trennen
engine.dispose()

# Ergebnis anzeigen (Dataframe -Auschnitt genügt- mit Ergebnis der Abfrage)
df_sub

,make,availablecars
0,Mercedes,125
1,Toyota,118
2,BMW,115
3,Chevrolet,110
4,Ford,105
5,Honda,99


#### <b>Aufgabe (5): Ermitteln Sie, wie viele Fahrzeuge jeder Verkäufer verkauft hat und zu welchem Gesamtpreis.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Informationen in den Tabellen 'Salespersons' und 'Sales'.
- Die Ergebnistabelle muss die Anzahl der verkauften Fahrzeuge, den Gesamtpreis der Verkäufe sowie den Namen, Vornamen, Tel.nr. und Email der Verkäufer enthalten.
- Die Ergebnistabelle muss nach dem gerundeten Gesamtpreis absteigend sortiert sein.



In [19]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Abfrage erstellen und in DataFrame speichern
df_sub = pd.read_sql_query("""select 
	sp.firstname,
	sp.lastname,
	sp.phone,
	sp.email,
	count(s.saleid) as totalcars,
	ROUND(CAST(SUM(s.saleprice) AS NUMERIC), 2) AS totalsales
from sales s
join salespersons sp on s.salespersonid = sp.salespersonid
group by sp.firstname, sp.lastname, sp.phone, sp.email
order by totalsales DESC;""",
                          con=engine)

# Datenbankverbindung trennen
engine.dispose()

# Ergebnis anzeigen (Dataframe -Auschnitt genügt- mit Ergebnis der Abfrage)
df_sub

,firstname,lastname,phone,email,totalcars,totalsales
0,Heather,Brown,+41 (0)46 288 75 19,heather.brown@example.com,261,8741578.45
1,Lauren,Robinson,+41 (0)71 214 12 13,lauren.robinson@example.com,260,8413807.47
2,Christine,Harris,+41 (0)61 523 86 58,christine.harris@example.com,259,8392149.63
3,Karen,Randall,+41 (0)66 616 17 40,karen.randall@example.com,263,8333139.56
4,Brian,Klein,+41 (0)77 449 26 62,brian.klein@example.com,254,8309576.34
5,Robert,Lyons,+41 (0)40 422 23 33,robert.lyons@example.com,248,8305445.39
6,Cathy,Becker,+41 (0)23 263 98 44,cathy.becker@example.com,251,7775226.86
7,Michael,Zhang,+41 (0)63 311 51 92,michael.zhang@example.com,244,7678064.65
8,Ryan,Baxter,+41 (0)40 706 53 70,ryan.baxter@example.com,236,7460279.34
9,Nicole,Carter,+41 (0)77 266 76 74,nicole.carter@example.com,224,7305422.86


##### <b>Aufgabe (6): Ermitteln Sie, welche Fahrzeuge im Januar 2024 verkauft wurden und zu welchem Verkaufspreis.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Informationen in den Tabellen 'Sales', 'Cars', 'Customers' und 'Salespersons'.
- Die Ergebnistabelle muss die CarID, Marke, Modell, Verkaufspreis, Verkaufsdatum sowie die Vornamen und Namen der Kunden und Verkäufer beinhalten.
- Limitieren Sie die Anzahl der mit der SQL Abfrage zurückgegebenen Fahrzeuge auf 10.


In [20]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Abfrage erstellen und in DataFrame speichern
df_sub = pd.read_sql_query("""select 
	cs.carid,
	cs.make,
	cs.model,
	saleprice,
	saledate,
	c.firstname,
	c.lastname,
	sp.firstname,
	sp.lastname
from sales s
join customers c on s.customerid = c.customerid
join salespersons sp on s.salespersonid = sp.salespersonid
join cars cs on s.carid = cs.carid
where saledate between '2024-01-01' and '2024-12-31'
and status = 'Sold'
limit 10;""",
                          con=engine)

# Datenbankverbindung trennen
engine.dispose()

# Ergebnis anzeigen (Dataframe -Auschnitt genügt- mit Ergebnis der Abfrage)
df_sub

,carid,make,model,saleprice,saledate,firstname,lastname,firstname,lastname
0,CMDGVZ1Q3X,Ford,Mustang,30586.68,2024-01-12,Logan,Fritz,Christine,Harris
1,CU5PRUA518,Toyota,Camry,29331.60,2024-01-08,Matthew,Melendez,Nicole,Carter
2,CRHJSJ3XVF,Toyota,Corolla,28889.39,2024-01-08,Randy,Jennings,Brian,Klein
3,CGYCABBZYE,Honda,Accord,38598.70,2024-02-27,Ashley,Jones,Robert,Lyons
4,CNMK2F55KP,BMW,320i,43982.24,2024-06-26,Kristopher,Lowe,Brian,Klein
5,COH474GZS4,Toyota,Camry,46803.02,2024-02-03,Julie,Thomas,Heather,Brown
6,CFCPDZQ8WH,Mercedes,E-Class,43558.91,2024-01-24,Mary,Stewart,Heather,Brown
7,CF3DIPUJ5R,Ford,Mustang,17086.98,2024-01-02,Brian,Cox,Michael,Zhang
8,CGQLKFPRL6,BMW,X5,47605.89,2024-03-09,Paul,York,Brian,Klein
9,CQYMCWTARQ,Mercedes,C-Class,16969.43,2024-08-14,Nicole,Miller,Ryan,Baxter


#### <b>Aufgabe (7): Ermitteln Sie die durchschnittlichen Servicekosten pro Mechaniker.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Informationen in den Tabellen 'Mechanics' und 'ServiceRecords'.
- Die Ergebnistabelle muss die Kontaktdaten (Vorname, Nachname, Telefonnummer, E-Mail) der Mechaniker enthalten
- Die Ergebnistabelle muss nach den durchschnittlichen Servicekosten absteigend sortiert sein.
- Die durchschnittlichen Servicekosten müssen auf zwei Nachkommastellen gerundet werden (z.B. 576.84).
- Die Ergebnistabelle muss nach den durchschnittlichen Servicekosten absteigend sortiert sein.


In [21]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Abfrage erstellen und in DataFrame speichern
df_sub = pd.read_sql_query("""select 
	m.firstname,
	m.lastname,
	m.phone,
	m.email,
	ROUND(CAST(AVG(cost) AS NUMERIC), 2) AS averageserviceprice
from servicerecords s
join mechanics m on s.mechanicid = m.mechanicid
group by m.firstname,
	m.lastname,
	m.phone,
	m.email
order by averageserviceprice DESC;""",
                          con=engine)

# Datenbankverbindung trennen
engine.dispose()

# Ergebnis anzeigen (Dataframe -Auschnitt genügt- mit Ergebnis der Abfrage)
df_sub

,firstname,lastname,phone,email,averageserviceprice
0,Shannon,Larsen,+41 (0)41 390 74 67,shannon.larsen@example.com,383.88
1,Melissa,Miller,+41 (0)77 431 12 29,melissa.miller@example.com,374.70
2,Cynthia,Pittman,+41 (0)39 208 58 20,cynthia.pittman@example.com,366.16
3,Bryan,White,+41 (0)23 791 68 46,bryan.white@example.com,363.38
4,Tiffany,Wilson,+41 (0)22 475 52 27,tiffany.wilson@example.com,361.28
5,Bradley,Mckinney,+41 (0)23 859 32 52,bradley.mckinney@example.com,360.99
6,Lisa,Hall,+41 (0)56 904 84 24,lisa.hall@example.com,359.72
7,Erin,Williamson,+41 (0)65 305 62 20,erin.williamson@example.com,356.38
8,Courtney,Ford,+41 (0)50 651 77 58,courtney.ford@example.com,351.50
9,Cassandra,Sanford,+41 (0)26 322 84 52,cassandra.sanford@example.com,344.29


#### <b>Aufgabe (8): Ermitteln Sie, welche <u>nicht</u> verkauften Fahrzeuge, wie oft im Service waren und welche Servicekosten dabei entstanden sind.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Informationen in den Tabellen 'Cars', 'Sales' und 'ServiceRecords'.
- Erstellen Sie eine Ergebnistabelle, welche die CarID, Marke, Modell, Baujahr sowie die Anzahl der Services und Summe der Servicekosten für jedes Fahrzeug enthält.
- Die Ergebnistabelle muss nach den Servicekosten absteigend sortiert sein.
- Limitieren Sie die Anzahl der mit der SQL Abfrage zurückgegebenen Fahrzeuge auf 10.


In [22]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Abfrage erstellen und in DataFrame speichern
df_sub = pd.read_sql_query("""select 
	c.carid,
	c.make,
	c.model,
	c.year,
	count(sr.carid) as serviceamount,
	sum(sr.cost) as totalcost
from servicerecords sr
join cars c on sr.carid = c.carid
join sales s on sr.carid = s.carid
where status not in ('Sold')
group by c.carid,
	c.make,
	c.model,
	c.year
order by totalcost DESC
LIMIT 10""",
                          con=engine)

# Datenbankverbindung trennen
engine.dispose()

# Ergebnis anzeigen (Dataframe -Auschnitt genügt- mit Ergebnis der Abfrage)
df_sub

,carid,make,model,year,serviceamount,totalcost
0,CMDLO9VN7K,Toyota,Camry,2019,27,19215.0
1,CEKZM11H3Q,Chevrolet,Impala,2020,32,16952.0
2,CC5U9IZYTU,Honda,Civic,2019,27,16500.0
3,C0F2AL9TXT,Toyota,Camry,2020,50,14655.0
4,C09R6HIZKV,Mercedes,E-Class,2023,20,13185.0
5,CA7UZQXOA6,Toyota,Camry,2023,44,13088.0
6,CPV2AUSUIO,Mercedes,C-Class,2023,24,12114.0
7,C4NID31AE4,BMW,X5,2020,15,11937.0
8,C8L0RP8L0C,Ford,Fiesta,2020,21,11403.0
9,CBJ5XPYX6W,Ford,Mustang,2021,20,10980.0


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [23]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('IP Address:', socket.gethostbyname(socket.gethostname()))
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.5.0-1025-azure
Datetime: 2024-10-07 09:49:31
Python Version: 3.11.10
IP Address: 172.18.0.2
-----------------------------------
